# Challenge

We decided to do the pcap scrapying in python with a library called "scapy", we discovered later that this library [does not support MQTT 5.0](https://github.com/secdev/scapy/pull/3292) packets and they are not parsed correctly, but we wrote custom code to solve this problem, and double checked everything in wireshark.

Libraries needed for the analysis

In [1]:
import pandas as pd
from scapy.all import *
from scapy.contrib.coap import *
from scapy.contrib.mqtt import *
from collections import Counter

In [ ]:
packets = rdpcap('capture.pcapng')

CQ1) How many different Confirmable PUT requests obtained an unsuccessful response from the local CoAP server?

We searched for confimable PUT requests which got an unsuccessful response from the local CoAP server and counted all different tokens.

In [3]:
put_con_packets = [pkt for pkt in packets if pkt.haslayer(CoAP) and pkt[CoAP].code == 3 and pkt[CoAP].type == 0]

uns_resp = [pkt for pkt in packets if pkt.haslayer(CoAP) and pkt[CoAP].code > 69 and pkt["IP"].src == "127.0.0.1"]

count = len({pkt[CoAP].token for pkt in put_con_packets} & {pkt[CoAP].token for pkt in uns_resp})

print(f"Answer: {count}")

Answer: 22


CQ2) How many CoAP resources in the coap.me public server received the same number of unique Confirmable and Non Confirmable GET requests?
> Assuming a resource receives X different CONFIRMABLE requests and Y different NONCONFIRMABLE  GET requests, how many resources have X=Y, with X>0?

We first looked at the DNS packets to get the IP address of the coap.me public server, than we counted how many confirmable and non confirmable GET requests each resource got and counted how many got the same number.

In [4]:
con_resources = Counter()
non_resources = Counter()

for pkt in packets: 
    if pkt.haslayer(CoAP) and pkt[CoAP].code == 1 and pkt["IP"].dst == "134.102.218.18":
        uri_path_str = ""
        for option in pkt[CoAP].options:
            if option[0] == "Uri-Path":
                uri_path = option[1].decode('utf-8')
                if uri_path:
                    uri_path_str = uri_path_str + "/" + uri_path
        if pkt[CoAP].type == 0:    
            con_resources[uri_path_str]+=1
        else:
            non_resources[uri_path_str]+=1
            

resources = [r for r in (set(con_resources.keys()) & set(non_resources.keys())) if con_resources[r] == non_resources[r]]

print(resources)

print(f"Answer: {len(resources)}")

['/validate', '/secret', '/large']
Answer: 3


CQ3) How many different MQTT clients subscribe to the public broker HiveMQ using multi-level wildcards?

As before, we first looked at the DNS packets to get HiveMQ IP addresses, then looked in every subscribe request for the multi level wildcard, when found we added the client (identified by its port) to the list.

In [5]:
clients = []

for pkt in packets:
    if pkt.haslayer(MQTT) and pkt[MQTT].type == 8 and (pkt.dst == "35.158.43.69" or pkt.dst == "35.158.34.213" or pkt.dst == "18.192.151.104"):
        if len(pkt[MQTT].topics) == 1:
            if "#" in pkt[MQTT].topics[0].topic.decode("utf-8") and pkt[TCP].sport not in clients:
                clients.append(pkt[TCP].sport)
        else: # MQTT 5.0 packets
            if "#" in pkt[MQTT].topics[1].load.decode("utf-8") and pkt[TCP].sport not in clients:
                clients.append(pkt[TCP].sport)

print(clients)
print(f"Answer: {len(clients)}")

[38641, 38619, 54449, 57863]
Answer: 4


CQ4) How many different MQTT clients specify a last Will Message to be directed to a topic having as first level "university"?

We searched every connect packet with a last will and checked if the topic started with university. If it did we added it to the list.

In [6]:
clients = []

for pkt in packets:
    if pkt.haslayer(MQTT) and pkt[MQTT].type == 1:
        try:
            if pkt[MQTT].willflag == 1 and pkt[MQTT].willtopic.decode("utf-8").startswith("university/") and pkt[TCP].sport not in clients:
                clients.append(pkt[TCP].sport)
        except: # MQTT 5.0 packets
            mqtt_payload = pkt[MQTT].load
            if len(mqtt_payload) > 7:  # Byte containing the flag need to exist
                connect_flags = mqtt_payload[7]
                will_flag = (connect_flags & 0b00000100) >> 2  # Extract the third bit
                if will_flag == 1:
                    # After connect flags
                    index = 8
                    # Skip keep alive
                    index += 2
                    # Read property len
                    property_length = mqtt_payload[index]
                    # Skip properties
                    index += 1 + property_length
                    # Read client id len
                    client_id_len = struct.unpack("!H", mqtt_payload[index:index+2])[0]
                    # Skip client id
                    index += 2 + client_id_len
                    # Check will properties
                    will_property_length = mqtt_payload[index]
                    # Skip will properties
                    index += 1 + will_property_length 
                    # Find will topic len
                    will_topic_len = struct.unpack("!H", mqtt_payload[index:index+2])[0]
                    # Move index to start of topic
                    index += 2
                    # Read topic
                    will_topic = mqtt_payload[index:index+will_topic_len].decode()

                    if will_topic.startswith("university/") and pkt[TCP].sport not in clients:
                        clients.append(pkt[TCP].sport)

print(clients)
print(f"Answer: {len(clients)}")

[38083]
Answer: 1


CQ5) How many MQTT subscribers receive a last will message derived from a subscription without a wildcard?

We search for every will topic in the packets. Than we look in the subscribe packets for matching subscribtion and we will find only subscribtions without a wildcard because we are matching explicitly with the will topic.

In [ ]:
topics = []
subscribers = []

for pkt in packets:
    if pkt.haslayer(MQTT) and pkt[MQTT].type == 1:
        try:
            if pkt[MQTT].willflag == 1:
                will_topic = pkt[MQTT].willtopic.decode("utf-8")
        except: # MQTT 5.0 packets
            mqtt_payload = pkt[MQTT].load
            if len(mqtt_payload) > 7:
                connect_flags = mqtt_payload[7]
                will_flag = (connect_flags & 0b00000100) >> 2
                if will_flag == 1:
                    index = 8
                    index += 2
                    property_length = mqtt_payload[index]  
                    index += 1 + property_length
                    client_id_len = struct.unpack("!H", mqtt_payload[index:index+2])[0]
                    index += 2 + client_id_len
                    will_property_length = mqtt_payload[index]
                    index += 1 + will_property_length
                    will_topic_len = struct.unpack("!H", mqtt_payload[index:index+2])[0]
                    index += 2
                    will_topic = mqtt_payload[index:index+will_topic_len].decode()
        if will_topic not in topics:
                    topics.append(will_topic)

print(topics)

for pkt in packets:
    if pkt.haslayer(MQTT) and pkt[MQTT].type == 8:
        if len(pkt[MQTT].topics) == 1:
            if pkt[MQTT].topics[0].topic.decode("utf-8") in topics and pkt[TCP].sport not in subscribers:
                subscribers.append(pkt[TCP].sport)
        else: # MQTT 5.0 packets
            if pkt[MQTT].topics[1].load.decode("utf-8") in topics and pkt[TCP].sport not in subscribers:
                subscribers.append(pkt[TCP].sport)

print(subscribers)

print(f"Answer: {len(subscribers)}")

['university/department12/room1/temperature', 'metaverse/room2/floor4', 'hospital/facility3/area3', 'metaverse/room2/room2']
[39551, 53557, 41789]
Answer: 3


CQ6) How many MQTT publish messages directed to the public broker mosquitto are sent with the retain option and use QoS “At most once”?

We first looked at the DNS packets to find the IP of the public broker and then we simply counted the matching messages.

In [8]:
count = 0
for pkt in packets:
    if pkt.haslayer(MQTT) and pkt[MQTT].type == 3 and pkt.dst == "5.196.78.28" and pkt[MQTT].RETAIN == 1 and pkt[MQTT].QOS == 0:
        count += 1

print(f"Answer: {count}")

Answer: 208


CQ7) How many MQTT-SN messages on port 1885 are sent by the clients to a broker in the local machine?

We looked for UDP packets with that port but found nothing (to be sure we also gave a fast scan at all udp packets in wireshark).

In [9]:
count = 0
for pkt in packets:
    if pkt.haslayer(UDP) and pkt[UDP].dport == 1885:
        count += 1

print(f"Answer: {count}")

Answer: 0
